<a href="https://colab.research.google.com/github/AAdewunmi/Breast-Cancer-Risk-Prediction-Project/blob/main/Breast_Cancer_Risk_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Breast Cancer Prediction Project

We will build a Deep Learning Model that is to be trained on breast cancer autopsy image dataset to predict if a person has breast cancer and if it is benigh or malign.

We will be using DenseNet-201 which is a convolution neural network that is 201 layers deep. We can load a pretrained version of the network or we can also retrain the model, which is what we are doing in this project.

We will then implement our very own website that will be built using Django framwork and host it on AWS.